In [2]:
import xarray as xr
import numpy as np
import pandas as pd
import dask.array
from zarr.storage import FSStore
import fsspec

In [3]:
fs = fsspec.filesystem(
    protocol="az",
    account_name="sauchdev",
    anon=False,
)

remote_store = FSStore(
    url="az://data/spot_prices",
    fs=fs,
)

In [6]:
fs.download("az://data/zarr_templates/spot_prices_sample.nc", "spot_prices_sample_001.nc")

sample = xr.open_dataset("spot_prices_sample_001.nc")

In [7]:
template = xr.Dataset(None, coords=sample.coords)

In [9]:
for name in template._coord_names:
    if np.issubdtype(template[name].dtype, np.datetime64):
        template[name].encoding["units"] = "hours since 2023-01-01 00:00:00"
    if np.issubdtype(template[name].dtype, np.timedelta64):
        template[name].encoding["units"] = "hours"

In [10]:
time_utc_idx = pd.date_range(start="2023-01-01T00:00:00", end="2026-01-01T00:00:00", freq=pd.Timedelta(hours=1))

In [11]:
template = template.reindex(time_utc=time_utc_idx)

In [12]:
shape = tuple(v for v in template.coords.sizes.values())

In [13]:
dims = tuple(k for k in template.dims.keys())

/tmp/ipykernel_9690/3042545308.py:1: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  dims = tuple(k for k in template.dims.keys())


In [14]:
empty_array = dask.array.empty(shape=shape, chunks=[shape[0], shape[1]])

In [15]:
for variable in sample.data_vars:
    template[variable] = (dims, empty_array)

In [18]:
template.to_zarr(
    store=remote_store,
    compute=False,
)

Delayed('_finalize_store-285af43a-7342-4750-9f89-09baeff374d0')